In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
# Define the model class
class LinearRegressor(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearRegressor, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        return self.linear(x)



In [22]:
# Example Data (Replace with your actual data)
# X_train: tensor of shape [n_samples, n_features]
# y_train: tensor of shape [n_samples, 1]
url = '/shared/share_mala/yuanzhe/adaptive_sampling/pipeline_datasets/'
csv_file = 'input_dim_1_train_init_data_mean_0.0ln_1.0sig_0.1no.2000.csv'

df = pd.read_csv(url + csv_file)
X_train = np.array(df[['Column0']])
y_train = np.array(df[['EVENT_LABEL']])  


# Convert data to PyTorch tensors if they aren't already
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)

# Hyperparameters
epochs = 100  # Number of training iterations
learning_rate = 0.01

# Model, Loss and Optimizer
model = LinearRegressor(input_size=X_train.shape[1], output_size=1)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    # Forward pass: Compute predicted y by passing X to the model
    y_pred = model(X_train)

    # Compute and print loss
    loss = criterion(y_pred, y_train)
    if epoch % 10 == 0:  # Print every 10th epoch
        print(f'Epoch {epoch}, Loss: {loss.item()}')

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

#torch.save(model, url + 'predictor.pkl')

model_scripted = torch.jit.script(model) # Export to TorchScript
model_scripted.save(url + 'predictor.pt') # Save

#https://stackoverflow.com/questions/55488795/unpickling-saved-pytorch-model-throws-attributeerror-cant-get-attribute-net

Epoch 0, Loss: 0.7675848603248596
Epoch 10, Loss: 0.4868123531341553
Epoch 20, Loss: 0.30910196900367737
Epoch 30, Loss: 0.19653074443340302
Epoch 40, Loss: 0.12515965104103088
Epoch 50, Loss: 0.07986758649349213
Epoch 60, Loss: 0.05109671503305435
Epoch 70, Loss: 0.03280135616660118
Epoch 80, Loss: 0.02115435153245926
Epoch 90, Loss: 0.013731011189520359


In [23]:
model(X_train)

tensor([[-1.1991e-01],
        [-9.0208e-03],
        [ 7.8658e-02],
        [ 1.7782e-01],
        [ 1.3394e-02],
        [ 3.5693e-02],
        [ 6.0576e-02],
        [-7.1788e-02],
        [-2.8497e-02],
        [ 1.5148e-02],
        [ 1.4714e-01],
        [-1.3423e-01],
        [ 1.9169e-01],
        [ 7.8642e-02],
        [-7.5718e-02],
        [-1.0296e-01],
        [ 2.2590e-02],
        [ 1.9364e-01],
        [ 8.9241e-02],
        [-1.2398e-01],
        [-3.5294e-02],
        [ 6.7666e-02],
        [ 4.6254e-02],
        [-1.2408e-01],
        [-5.6806e-02],
        [ 8.7634e-02],
        [ 1.2648e-02],
        [-1.0262e-01],
        [ 3.3334e-02],
        [ 4.5507e-02],
        [-5.1666e-02],
        [ 1.1486e-01],
        [-3.9325e-02],
        [-2.7642e-02],
        [ 7.2447e-03],
        [ 4.7092e-02],
        [ 5.3496e-02],
        [-1.8419e-01],
        [ 3.9842e-02],
        [-6.8303e-02],
        [ 7.1846e-02],
        [ 8.3227e-02],
        [ 2.2976e-02],
        [-1